# Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import pathlib
from PIL import Image
import ast
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# [OPTIONAL] Working with DICOM files

In [ ]:
# Install libraries for DICOM files

# #!wget 'https://anaconda.org/conda-forge/gdcm/2.8.9/download/linux-64/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -q
# #!conda install 'gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
# !conda install -c conda-forge gdcm -y
# !conda install -c conda-forge pydicom -y

In [ ]:
# import pydicom
# from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# CONVERTING DICOM FILES TO NP ARRAYS PROPERLY
# Ref : https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way

# def dicom2arr(path, voi_lut = True, fix_monochrome = True):

#     dicom = pydicom.read_file(path)
    
#     # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
#     # "human-friendly" view
#     if voi_lut:
#         arr = apply_voi_lut(dicom.pixel_array, dicom)
#     else:
#         arr = dicom.pixel_array
    
#     # depending on this value, X-ray may look inverted - fix that:
#     if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
#         arr = np.amax(arr) - arr
        
#     arr = arr - np.min(arr)
#     arr = arr / np.max(arr)
#     arr = (arr * 255).astype(np.uint8)
        
#     return arr

# [OPTIONAL] Convert dcm to jpg
I did this for two reasons:
1. Because TF OD API explicitly states so in their docs:
"Dataset Requirements
For every example in your dataset, you should have the following information: An RGB image for the dataset **encoded as jpeg or png**.
2. Because it's easier to work with jpg's.

In [ ]:
# def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
#     # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
#     im = Image.fromarray(array)
    
#     if keep_ratio:
#         im.thumbnail((size, size), resample)
#     else:
#         im = im.resize((size, size), resample)
    
#     return im

In [ ]:
# image_id = []
# dim0 = []
# dim1 = []
# splits = []

# for split in ['test', 'train']:
#     save_dir = f'/kaggle/tmp/{split}/'

#     os.makedirs(save_dir, exist_ok=True)
    
#     for dirname, _, filenames in os.walk(f'../input/siim-covid19-detection/{split}'):
#         for file in filenames:
#             # set keep_ratio=True to have original aspect ratio
#             xray = dicom2arr(os.path.join(dirname, file))
#             im = resize(xray, size=256)  
#             im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))

#             image_id.append(file.replace('.dcm', ''))
#             dim0.append(xray.shape[0])
#             dim1.append(xray.shape[1])
#             splits.append(split)

In [ ]:

# for dirname, _, filenames in os.walk('../input/siim-covid19-resized-to-256px-jpg/train'):
#     for filename in filenames:
#         #print(dirname, filename)
#         path = os.path.join(dirname, filename)
#         #print(path)
#         img= Image.open(path)  
#         image = np.array(img)
#         print(image)
#         print(image.shape)
#         img_arr = dicom2arr(path)
#         print(img_arr)
#         print(np.max(img_arr))
#         print(img_arr.shape)
#         print(type(img_arr))
        


# Install TF Object detection API

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.17.2/protoc-3.17.2-linux-x86_64.zip -q
!unzip -o protobuf.zip
!rm protobuf.zip

In [ ]:
%cd /kaggle/working/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
#run model builder test
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# The following two imports are for creating TFRecord files
from object_detection.utils import dataset_util
from object_detection.dataset_tools import tf_record_creation_util

# Create annotations

In [ ]:
!mkdir /kaggle/working/Annotations
%cd /kaggle/working/

In [ ]:
%%writefile Annotations/label_map.pbtxt
item {
    id: 1
    name: 'negative for pneumonia'
}

item {
    id: 2
    name: 'typical'
}

item {
    id: 3
    name: 'indeterminate'
}

item {
    id: 4
    name: 'atypical'
}

# Working with bboxes and labels

In [ ]:
import pandas as pd
!pwd
path = '../input/siim-covid19-detection/'
train_image = pd.read_csv(path+'train_image_level.csv')
train_study = pd.read_csv(path+'train_study_level.csv')
sample_submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
print(len(train_image))
print(len(train_study))

NOTES: 
* Some of the values in the **StudyInstanceUID** column in **train_image** are related (point) to several images - therefore there are 6334 images but only 6054 StudyInstanceUID's. (Does it mean that multiple images have identical bboxes and labels and this was done just to save time and space for identical boxes/labels?)
* **StudyInstanceUID** column in **train_image** are identical to the **id** column in **train_study** (but without the _study suffix)

In [ ]:
train_study.head()

In [ ]:
train_study['id'] = train_study['id'].str.replace('_study', '')


In [ ]:
train_study.head()

In [ ]:
train_study.rename(columns={'id': 'StudyInstanceUID'}, inplace=True)

In [ ]:
train_study.head()

In [ ]:
train_image.info()
print('\n')
train_study.info()

In [ ]:
train_result = train_image.merge(train_study, on='StudyInstanceUID', how='left')

In [ ]:
train_result.head()

In [ ]:
train_result.describe()

In [ ]:
train_result['id'] = train_result['id'].str.replace('_image', '')

In [ ]:
original_dims = pd.read_csv('../input/siim-covid19-resized-to-256px-jpg/meta.csv')

In [ ]:
original_dims.head()

In [ ]:
original_dims.rename(columns={'image_id': 'id'}, inplace=True)

In [ ]:
train_result_final = train_result.merge(original_dims, on='id', how='left')

In [ ]:
train_result_final.head()

# Scale bboxes proportionally

In [ ]:
train_result_final["boxes"] = train_result_final["boxes"].fillna("[{'x':0, 'y':0, 'width':1, 'height':1}]")

In [ ]:
import ast
train_result_final["boxes"] = train_result_final["boxes"].apply(lambda x: ast.literal_eval(x))

In [ ]:
train_result_final.head()

In [ ]:
def unpack_bboxes(df):
    """ go from xmin,ymin,width,height --> xmin,ymin,xmax,ymax """
    for dictionary in df["boxes"]:
        df["xmin"] = dictionary["x"]
        df["ymin"] = dictionary["y"]
        df["xmax"] = dictionary["x"] + dictionary["width"]
        df["ymax"] = dictionary["y"] + dictionary["height"]
    return df

In [ ]:
def scale_bbox_coor(df):
    if df['xmin'] != 0:
        df['xmin'] *= (256 / df['dim1'])
        df['xmax'] *= (256 / df['dim1'])
        df['ymin'] *= (256 / df['dim0'])
        df['ymax'] *= (256 / df['dim0'])
    return df

In [ ]:
print("Unpacking bboxes into separate columns. This will take ~20 secs")
train_result_final = train_result_final.apply(unpack_bboxes, axis=1)

In [ ]:
train_result_final = train_result_final.apply(scale_bbox_coor, axis=1)

In [ ]:
train_result_final.head()

In [ ]:
labels_dict = {'Negative for Pneumonia': ["negative", 1], 'Typical Appearance': ["typical", 2], 'Indeterminate Appearance': ["indeterminate", 3], 'Atypical Appearance': ["atypical", 4]}

In [ ]:
def convert_and_combine_classes(df):
    for lbl in labels_dict:
        if df[lbl]:
            df['class'] = labels_dict[lbl][0]
            df['class_num'] = labels_dict[lbl][1]
    return df

In [ ]:
train_result_final = train_result_final.apply(convert_and_combine_classes, axis=1)

In [ ]:
train_result_final.head()

# Split train DataFrame into train/validation

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_result_final, test_size=0.1)

In [ ]:
train_df = train_df.filter(['id','class', 'class_num', 'xmin', 'ymin', 'xmax', 'ymax'], axis=1)
val_df = val_df.filter(['id','class', 'class_num', 'xmin', 'ymin', 'xmax', 'ymax'], axis=1)

In [ ]:
train_df.head()

In [ ]:
val_df.head()

# Split train images into train/validation datasets

In [ ]:
# Move 10% of images from train to validation directory
!pwd
!mkdir -p /kaggle/working/dataset/train
!mkdir -p /kaggle/working/dataset/validation

import os
import shutil

original_dataset_path = '../input/siim-covid19-resized-to-256px-jpg/train'

def copy_split_dataset(df, split):
    for _, row in df.iterrows():
        source_file_path = os.path.join(original_dataset_path, (row['id'] + '.jpg'))
        dest_file_path = os.path.join(('./dataset/' + split), (row['id'] +'.jpg'))
        shutil.copy(source_file_path, dest_file_path)

In [ ]:
copy_split_dataset(train_df, 'train')
copy_split_dataset(val_df, 'validation')


In [ ]:
print(len(os.listdir('/kaggle/working/dataset/train/')))
print(len(os.listdir('/kaggle/working/dataset/validation/')))

In [ ]:
train_df.to_csv('train_df.csv', encoding='utf-8')
val_df.to_csv('val_df.csv', encoding='utf-8')

# Create TFRecord files

In [ ]:
import io
from collections import namedtuple

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [ ]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename + '.jpg')), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'])
        xmaxs.append(row['xmax'])
        ymins.append(row['ymin'])
        ymaxs.append(row['ymax'])
        classes_text.append(row['class'].encode('utf8'))
        classes.append(row['class_num'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        #'image/height': dataset_util.int64_feature(height),
        #'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        #'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
df_to_dirnames_dict = {'train': train_df, 'validation': val_df}

dir_path = './dataset/'
for dirname in next(os.walk(dir_path))[1]:
    # if dirname != 'test':
    writer = tf.io.TFRecordWriter(f'tfrecord_{dirname}.tfrec')
    path = os.path.join(dir_path, dirname)
    examples = df_to_dirnames_dict[dirname]
    grouped = split(examples, 'id')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()

# Using pre-trained models from the zoo

In [ ]:
# !mkdir pre-trained-models
# !mkdir models/my_ssd_resnet50_v1_fpn

In [ ]:
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz  
# !tar xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz --directory=pre-trained-models

# %cd ~
# # !cp /kaggle/input/tfodpipelineconfig/pipeline.config /kaggle/working/models/my_ssd_resnet50_v1_fpn

In [ ]:
# !cp /kaggle/working/models/research/object_detection/model_main_tf2.py /kaggle/working/
# %cd /kaggle/working/
# !python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

# Using Keras

In [ ]:
def prepare_sample(features):
    image = tf.image.resize(features["image/encoded"], size=(256, 256))
    return image, features["image/object/class/label"]


def get_dataset(filenames, batch_size):
    dataset = (
        tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
        .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
        .shuffle(batch_size * 10)
        .batch(batch_size)
        .prefetch(AUTOTUNE)
    )
    return dataset


def parse_tfrecord_fn(example):
    feature_description = {
        "image/encoded": tf.io.FixedLenFeature([], tf.string),
        "image/format": tf.io.FixedLenFeature([], tf.string),
        "image/object/bbox/xmin": tf.io.FixedLenFeature([], tf.float32),
        "image/object/bbox/xmax": tf.io.FixedLenFeature([], tf.float32),
        "image/object/bbox/ymin": tf.io.FixedLenFeature([], tf.float32),
        "image/object/bbox/ymax": tf.io.FixedLenFeature([], tf.float32),   
        "image/object/class/text": tf.io.FixedLenFeature([], tf.string),
        "image/object/class/label": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image/encoded"] = tf.io.decode_jpeg(example["image/encoded"], channels=1)
#     example["image/object/bbox/xmin"] = tf.sparse.to_dense(example["image/object/bbox/xmin"])
#     example["image/object/bbox/xmax"] = tf.sparse.to_dense(example["image/object/bbox/xmax"])
#     example["image/object/bbox/ymin"] = tf.sparse.to_dense(example["image/object/bbox/ymin"])
#     example["image/object/bbox/ymax"] = tf.sparse.to_dense(example["image/object/bbox/ymax"])
    return example

# Explore one sample from the generated TFRecord

In [ ]:
raw_dataset = tf.data.TFRecordDataset("tfrecord_train.tfrec")
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)

for features in parsed_dataset.take(1):
    for key in features.keys():
        if key != "image/encoded":
            print(f"{key}: {features[key]}")

    print(f"Image shape: {features['image/encoded'].shape}")
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    # plt.figure(figsize=(7, 7))
    
    width = features['image/object/bbox/xmax'] - features['image/object/bbox/xmin']
    height = features['image/object/bbox/ymax'] - features['image/object/bbox/ymin']
    p = matplotlib.patches.Rectangle((features['image/object/bbox/xmin'], features['image/object/bbox/ymin']),
                                     width, height,
                                     ec='r', fc='none', lw=1.5)
    ax.add_patch(p)
    ax.imshow(features["image/encoded"].numpy())
    
    #plt.imshow(features["image/encoded"].numpy())
    plt.show()

In [ ]:
train_filenames = tf.io.gfile.glob("tfrecord_train.tfrec")
batch_size = 32
epochs = 1
steps_per_epoch = 50
AUTOTUNE = tf.data.experimental.AUTOTUNE

input_tensor = tf.keras.layers.Input(shape=(256, 256, 1), name="image/encoded")
model = tf.keras.applications.EfficientNetB0(
    input_tensor=input_tensor, weights=None, classes=4
)


model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)


model.fit(
    x=get_dataset(train_filenames, batch_size),
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
)


# Appendix

# Write all jpg's into a single 'dataset' numpy array

In [ ]:
# dataset = np.ndarray(shape=(len(train_image), 1, 256, 256), dtype=np.float32)

# for dirname, _, filenames in os.walk('../input/siim-covid19-resized-to-256px-jpg/train'):
#     i = 0
#     for filename in filenames:
#         path = os.path.join(dirname, filename)
#         img= Image.open(path)  
#         np_arr_image = np.array(img)
#         dataset[i] = np_arr_image
#         i += 1
        
#         if i % 500 == 0:
#             print(f"{i} images added to dataset")
#     print("All images added to dataset!")

In [ ]:
# dataset.shape